In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        np.random.seed(42)
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.bias_hidden = np.random.rand(hidden_size)
        self.bias_output = np.random.rand(output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward_propagation(self, inputs):
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)
        
        return self.hidden_output, self.final_output

    def backpropagation(self, inputs, hidden_output, final_output, expected_output, learning_rate):
        output_error = expected_output - final_output
        output_delta = output_error * self.sigmoid_derivative(final_output)
        
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(hidden_output)
        
        self.weights_hidden_output += hidden_output.T.dot(output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * learning_rate
        
        self.weights_input_hidden += inputs.T.dot(hidden_delta) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * learning_rate

    def train(self, inputs, expected_output, learning_rate, epochs):
        for epoch in range(epochs):
            hidden_output, final_output = self.forward_propagation(inputs)
            self.backpropagation(inputs, hidden_output, final_output, expected_output, learning_rate)

    def predict(self, inputs):
        _, final_output = self.forward_propagation(inputs)
        return final_output

    def accuracy(self, predictions, labels):
        pred_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(labels, axis=1)
        return np.mean(pred_labels == true_labels)

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nn = NeuralNetwork(input_size=4, hidden_size=5, output_size=3)
nn.train(X_train, y_train, learning_rate=0.1, epochs=10000)

test_results = nn.predict(X_test)
print("Test Accuracy:", nn.accuracy(test_results, y_test))


Test Accuracy: 0.9666666666666667
